In [1]:
# Import libraries
import cv2
import os
import numpy as np
from keras.models import load_model

In [2]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
gender_labels = ['Woman', 'Man']

In [3]:
# Folder path
emotion_model_path = os.path.abspath('models/emotion_model_1.h5')
gender_model_path = os.path.abspath('models/gender_model_es.h5')


In [4]:
# Load models
emotion_model = load_model(emotion_model_path)
gender_model = load_model(gender_model_path)

In [5]:
# Start the webcam feed
cap = cv2.VideoCapture(0)

In [6]:
# Pass video path
#cap = cv2.VideoCapture("video/michael.MOV")

In [7]:
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break

    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # Take each face available on the camera and preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (102, 255, 255), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]

        # For Grayscale Model (emotion_model is trained on grayscale images)
        cropped_img_gray = cv2.resize(roi_gray_frame, (48, 48))  # Resize to the model's input shape
        cropped_img_gray = np.expand_dims(cropped_img_gray, axis=-1)  # Add channel dimension
        cropped_img_gray = np.expand_dims(cropped_img_gray, axis=0)  # Add batch dimension

        # For RGB Model (gender_model is trained on RGB images)
        cropped_img_rgb = cv2.resize(roi_gray_frame, (96, 96))  # Resize to the model's input shape
        cropped_img_rgb = np.expand_dims(cropped_img_rgb, axis=-1)  # Add channel dimension (single channel)
        cropped_img_rgb = np.concatenate([cropped_img_rgb] * 3, axis=-1)  # Convert grayscale to 3 channels
        cropped_img_rgb = np.expand_dims(cropped_img_rgb, axis=0)  # Add batch dimension
    
         # Predict emotions using emotion_model
        emotion_prediction = emotion_model.predict(cropped_img_gray)
        maxindex_emotion = int(np.argmax(emotion_prediction))
        emotion_label = emotion_dict[maxindex_emotion]

        # Predict gender using gender_model
        gender_prediction = gender_model.predict(cropped_img_rgb)
        maxindex_gender = int(np.argmax(gender_prediction))
        gender_label = gender_labels[maxindex_gender]

        # Display output
        cv2.putText(frame, f'Emotion: {emotion_label}', (x+5, y-20), cv2.FONT_HERSHEY_DUPLEX, 1, (102, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Gender: {gender_label}', (x+5, y+20), cv2.FONT_HERSHEY_DUPLEX, 1, (102, 255, 255), 2, cv2.LINE_AA)
        
    # Display output
    cv2.imshow('Emotion and Gender Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

1/1 [==============================] - 0s 14ms/step


KeyboardInterrupt: 

In [8]:
# Release the resources used by the camera or video file
cap.release()

In [9]:
# Close all windows created by OpenCV
cv2.destroyAllWindows()